In [1]:
# Example of Pulling Json Infromation from Udemy API. 10-02-2022
import cred_user_pwd_keys # a .py file in the same folder that has account details
#DO NOT UPLOAD THE cred_user_pwd_keys.py file to GIT or any other file sharing site

# Requests is one of the easier to understand modules to download a website or many web related APIs
import requests


#I don't put much into pandas, but you can load info into pandas for normal manipulation as needed
import pandas as pd
# Although not scrictly needed most API calls will return a json format. 
# Python's dictionary and Json are very simlar and easy to swtich between.
import json
import re # used for pattern matching and removing bad characters




# MEDIUM DIFFICULITY USERID TOKEN AND KEY TOKEN
## Udemy Token

Authentication is handled in the header of the URL request

https://www.udemy.com/developers/affiliate/

In [2]:
# Authorization data
#https://www.udemy.com/developers/affiliate/

#https://www.udemy.com/developers/affiliate/methods/get-courses-list/
#https://www.udemy.com/developers/affiliate/methods/get-coursereviews-list/


#Needed to encode / decode the keys
import base64



consumer_key = cred_user_pwd_keys.Udemy_Client_id
consumer_secret = cred_user_pwd_keys.Udemy_Client_Secret
consumer_key_secret = consumer_key+":"+consumer_secret
consumer_key_secret_enc = base64.b64encode(consumer_key_secret.encode()).decode()

#print(consumer_key_secret_enc)

headersAuth = {
    "Accept": "application/json, text/plain, */*",
    'Authorization': 'Basic '+ str(consumer_key_secret_enc),
    "Content-Type": "application/json;charset=utf-8"
}

#varCourseID = '9407966' # 2022 Complete Python Bootcamp From Zero to Hero in Python
## Authentication request

#Use the websites above to help you define your searches
NumberOfResultsPerPage = 5 #I think this maxes out at 100
SeachTerm = 'Python 2022' #Spaces seem to work fine even if they are replace by %20 in the websites
Language = 'en' # 2 digit id


ManuallyDefinedUrl = f'https://www.udemy.com/api-2.0/courses/?page_size={NumberOfResultsPerPage}&search={SeachTerm}&language={Language}'

testWebGet = requests.get(ManuallyDefinedUrl, headers=headersAuth, verify=True)

print(testWebGet) # a 200 response means it worked and you got results

ParsedData = json.loads(testWebGet.text)

#View the result in a more human readable format
print(json.dumps(ParsedData, indent=4))

<Response [200]>
{
    "count": 7950,
    "next": "https://www.udemy.com/api-2.0/courses/?language=en&page=2&page_size=5&search=Python+2022",
    "previous": null,
    "results": [
        {
            "_class": "course",
            "id": 567828,
            "title": "2022 Complete Python Bootcamp From Zero to Hero in Python",
            "url": "/course/complete-python-bootcamp/",
            "is_paid": true,
            "price": "$139.99",
            "price_detail": {
                "amount": 139.99,
                "currency": "USD",
                "price_string": "$139.99",
                "currency_symbol": "$"
            },
            "price_serve_tracking_id": "Srcmc1XmT4SiVtBut0cKHQ",
            "visible_instructors": [
                {
                    "_class": "user",
                    "title": "Jose Portilla",
                    "name": "Jose",
                    "display_name": "Jose Portilla",
                    "job_title": "Head of Data Science at Pieri

In [3]:

#This is just the first page Lets print the course name and the instructors of the courses:
for Udemyclass in ParsedData['results']:
    print(Udemyclass['title'])
    for instructor in Udemyclass['visible_instructors']:
        print('   --', instructor['display_name'], " ID: ", instructor['url'])


2022 Complete Python Bootcamp From Zero to Hero in Python
   -- Jose Portilla  ID:  /user/joseportilla/
The Complete Python Course | Learn Python by Doing in 2022
   -- Codestars • over 2 million students worldwide!  ID:  /user/codestars/
   -- Jose Salvatierra  ID:  /user/josesalvatierra/
   -- Rob Percival  ID:  /user/robpercival/
   -- Teclado by Jose Salvatierra  ID:  /user/teclado-4/
The Python Mega Course: Learn Python in 40 Days with 20 Apps
   -- Ardit Sulce  ID:  /user/adiune/
Python for Absolute Beginners | Python Beginner to Pro 2022
   -- 365 Careers  ID:  /user/365careers/
   -- Giles McMullen-Klein  ID:  /user/gilesmcmullen/
100 Days of Code: The Complete Python Pro Bootcamp for 2023
   -- Dr. Angela Yu  ID:  /user/4b4368a3-b5c8-4529-aa65-2056ec31f37e/


In [4]:
#Next lets Use the courses we found the first 5 and get 2 pages of reviews for each course:

#First lets define a more generic function grabber for course reviews we can reference over and over
def GetCourseReviews(CourseID, page, page_size):
    url = f'https://www.udemy.com/api-2.0/courses/{str(CourseID)}/reviews/?page={str(page)}&page_size={str(page_size)}'

    
    consumer_key = cred_user_pwd_keys.Udemy_Client_id
    consumer_secret = cred_user_pwd_keys.Udemy_Client_Secret
    consumer_key_secret = consumer_key+":"+consumer_secret
    consumer_key_secret_enc = base64.b64encode(consumer_key_secret.encode()).decode()

    #print(consumer_key_secret_enc)

    headersAuth = {
        "Accept": "application/json, text/plain, */*",
        'Authorization': 'Basic '+ str(consumer_key_secret_enc),
        "Content-Type": "application/json;charset=utf-8"
    }

    testWebGet = requests.get(url, headers=headersAuth, verify=True)

    ParsedData = json.loads(testWebGet.text)
    #print(ParsedData)
    
    return ParsedData['results']
    #Another approach is to use the next URL part
    
    # But to minimize how many hits you get on the Udemy API you might want to stop early
    #Note* if we go to a page that has no results you will get the following result and not an error
    #{
          #"count": 6,
          #"next": null,
          #"previous": "https://www.udemy.com/api-2.0/courses/3996952/reviews/?page=4&page_size=5",
          #"results": []
    #}

    

    

In [5]:
Pagesize = 5
for Udemyclass in ParsedData['results']:
    CourseTitle = Udemyclass['title']
    CourseID = Udemyclass['id']
    FileName = re.sub('[^A-Za-z0-9]+', '', str(CourseID) + "_" + CourseTitle)
    
    Reviewlist = [] #List of all reviews in all the pages we loop through
    for i in range(3): #The number of pages of reviews to loop through
        print('Page ', i+1, ' of Review for: ' + CourseTitle)
        Reviewlist.append(GetCourseReviews(CourseID, i+1, Pagesize))
        
    #Export Results to a .json file.
    with open(f"{FileName}.json", 'w') as fp:
        json.dump(Reviewlist, fp)
        
    print(json.dumps(Reviewlist[2], indent=4))

Page  1  of Review for: 2022 Complete Python Bootcamp From Zero to Hero in Python
Page  2  of Review for: 2022 Complete Python Bootcamp From Zero to Hero in Python
Page  3  of Review for: 2022 Complete Python Bootcamp From Zero to Hero in Python
[
    {
        "_class": "course_review",
        "id": 131165944,
        "content": "",
        "rating": 5.0,
        "created": "2022-12-19T22:36:25-08:00",
        "modified": "2022-12-20T15:08:10-08:00",
        "user_modified": "2022-12-19T22:36:29-08:00",
        "user": {
            "_class": "user",
            "title": "Dimple Ahuja",
            "name": "Dimple",
            "display_name": "Dimple Ahuja"
        }
    },
    {
        "_class": "course_review",
        "id": 131164592,
        "content": "",
        "rating": 5.0,
        "created": "2022-12-19T22:16:06-08:00",
        "modified": "2022-12-20T15:08:10-08:00",
        "user_modified": "2022-12-19T22:16:07-08:00",
        "user": {
            "_class": "user",
   